In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
import os

_ = load_dotenv()

# Stubs

In [4]:
from tinygen.tool import Tool

find_tool = Tool(
    "find",
    """find [STR]
returns a list of paths that match given
STR. STR is a string""",
    lambda args: input(f'find({args}): '),
)

search_tool = Tool(
    "search",
    "search [QUERY] returns a list of paths to files containing that match QUERY. QUERY is a regular string",
    lambda args: input(f'search({args}): '),
)

open_tool = Tool(
    "open",
    """open [PATH]
returns contents of file at PATH.""",
    lambda args: input(f'open({args}): '),
)

write_tool = Tool(
    "write",
    """write [PATH, CONTENT]
write CONTENT to the file at PATH. This overwrites existing content if any.""",
    lambda args: input(f'write({args}): '),
)

tools = [find_tool, search_tool, open_tool, write_tool]

# Planning

- Plan + Self-Reflection: https://platform.openai.com/playground/p/jxYEE69LEHREHqf9e7I6LgGj


In [5]:
from tinygen.plan import plan

task = 'Add a new FastAPI endpoint for "/api/diff_me" that accepts githubUrl and prompt as query parameters'
the_plan = plan(tools, task, debug=True)

*** ASSISTANT:
Expert 1: We need to find the FastAPI application file first. We can use the find tool with a pattern that matches python files. The pattern can be something like ".*\.py$".

Expert 2: Yes, and once we find the FastAPI application file, we need to open it and find the place where we can add the new endpoint. We can use the open tool for this.

Expert 3: After finding the right place in the file, we need to add the new endpoint. We can use the write tool to add the new endpoint to the file. The new endpoint will be a function that accepts githubUrl and prompt as query parameters.

PLAN:
1. Use find tool with parameters [".*\.py$"] to find the FastAPI application file.
2. Use open tool with the path of the FastAPI application file to open it.
3. Use search tool with query "app = FastAPI()" to find the place where we can add the new endpoint.
4. Use write tool with the path of the FastAPI application file and the new endpoint code as parameters to add the new endpoint to th

In [6]:
print(the_plan)

PLAN:
1. Use find tool with parameters [".*\.py$"] to find the FastAPI application file.
2. Use open tool with the path of the FastAPI application file to open it.
3. Use search tool with query "app = FastAPI()" to find the place where we can add the new endpoint.
4. Use write tool with the path of the FastAPI application file and the new endpoint code as parameters to add the new endpoint to the file. The new endpoint code will be something like:
```python
@app.get("/api/diff_me")
def diff_me(githubUrl: str, prompt: str):
    # function body here
    pass
```


# Plan Execution
- Execute (React): https://platform.openai.com/playground/p/BZp876JtEj3ss8SLqI2Acs85

In [31]:
from tinygen.agent import CodeAgent

agent = CodeAgent(tools, task, the_plan, debug=True)

In [32]:
agent.run()

*** SYSTEM:
You exist in an environment where you can interact with the file system using distinct tools below:
    
find [PATTERN]
returns a list of paths that match given
PATTERN. PATTERN is a valid regular expression for Python's
re.compile.

search [QUERY] returns a list of paths to files containing that match QUERY. QUERY is a regular string

open [PATH]
returns contents of file at PATH.

write [PATH, CONTENT]
write CONTENT to the file at PATH. This overwrites existing content if any.

Given a programming task and a step-by-step sequence of actions using the tools described above, the objective is to complete the task.

The arguments passed to a tool should be a Python list of strings.

Each assistant response should be formatted as follows:

OBSERVATION: <Observation about preceding user message in context of accomplishing the programming task>
THOUGHT: <Thought on next step to accomplish programming task, following the step-by-step sequence of the plan.
ACTION: <Specifies one of

find(['.*\\.py$']):  ['main.py']


*** ASSISTANT:
OBSERVATION: There is a file named "main.py" which is likely the FastAPI application file.
THOUGHT: Next, I should open the file I found.
ACTION: open ["main.py"]



open(['main.py']):  from fastapi import FastAPI  app = FastAPI()   @app.get("/") async def root():     return {"message": "Hello World"}


*** ASSISTANT:
OBSERVATION: The main.py file contains the FastAPI application setup and at least one endpoint.
THOUGHT: I should search for the place where we can add the new endpoint.
ACTION: search ["app = FastAPI()"]



search(['app = FastAPI()']):  ['main.py']


*** ASSISTANT:
OBSERVATION: The FastAPI application instance is created in the "main.py" file.
THOUGHT: I should add the new endpoint to the file using the write tool.
ACTION: write ["main.py", 'from fastapi import FastAPI\n\napp = FastAPI()\n\n@app.get("/")\nasync def root():\n    return {"message": "Hello World"}\n\n@app.get("/api/diff_me")\ndef diff_me(githubUrl: str, prompt: str):\n    # function body here\n    pass\n']



write(['main.py', 'from fastapi import FastAPI\n\napp = FastAPI()\n\n@app.get("/")\nasync def root():\n    return {"message": "Hello World"}\n\n@app.get("/api/diff_me")\ndef diff_me(githubUrl: str, prompt: str):\n    # function body here\n    pass\n']):  OK


*** ASSISTANT:
OBSERVATION: The main.py file was updated with the new endpoint.
THOUGHT: I have updated the main.py file with the new endpoint. The task is complete.
ACTION: DONE



In [29]:
import tinygen.extract

msg = agent.messages[10]
tinygen.extract.extract_action(msg)

Action(name='DONE', args=[])

In [30]:
isinstance([], list)

True

# Github Integration


In [18]:
import tinygen.github

from importlib import reload
reload(tinygen)

gh_api_key = os.getenv("GITHUB_API_KEY")
repo_spec = 'https://github.com/leoshimo/copier_python_template'
gh = tinygen.github.GithubRepo(repo_spec, gh_api_key)

In [19]:
# Open
gh.open("/src/../README.md")

"# README\n\n[Copier](https://copier.readthedocs.io/) template for a simple Python projects\n\n- Python project with `src` and `test` managed by `poetry`\n- `notebooks` for Jupyter notebooks\n- `Makefile` as task runner. See `make help`\n\n## Prerequisites\n\n- `pyenv`\n- `poetry`\n- `copier`\n\n## Project Setup\n\n```sh\n$ copier copy gh:leoshimo/copier_python_template my_project\n$ cd my_project\n$ make setup\n```\n\n## Running Tasks\n\nSee `make help` for list of tasks.\n\n## API Keys\n\nAdd `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:\n\n```python\nimport os\nfrom dotenv import load_dotenv\nload_dotenv()\nos.environ.get('SOME_API_KEY')\n```\n\n"

In [20]:
# Find File
gh.find_file("READ")

['README.md']

In [21]:
gh.search("read")

['README.md', 'pyproject.toml.jinja']

# Workspace

In [22]:
import tinygen.workspace

wkspc = tinygen.workspace.Workspace(gh)

In [23]:
wkspc.find("app.py")

['src/{{ project_name }}/app.py.jinja']

In [24]:
wkspc.search("READ")

['README.md', 'pyproject.toml.jinja']

In [25]:
wkspc.open("README.md")

"# README\n\n[Copier](https://copier.readthedocs.io/) template for a simple Python projects\n\n- Python project with `src` and `test` managed by `poetry`\n- `notebooks` for Jupyter notebooks\n- `Makefile` as task runner. See `make help`\n\n## Prerequisites\n\n- `pyenv`\n- `poetry`\n- `copier`\n\n## Project Setup\n\n```sh\n$ copier copy gh:leoshimo/copier_python_template my_project\n$ cd my_project\n$ make setup\n```\n\n## Running Tasks\n\nSee `make help` for list of tasks.\n\n## API Keys\n\nAdd `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:\n\n```python\nimport os\nfrom dotenv import load_dotenv\nload_dotenv()\nos.environ.get('SOME_API_KEY')\n```\n\n"

In [26]:
wkspc.write("README.md", "Hello world!")
wkspc.write("src/{{ project_name }}/app.py.jinja", "print('Hello world')")

In [27]:
print(wkspc.diff())

--- a/README.md
+++ b/README.md
@@ -1,37 +1 @@
-# README
-
-[Copier](https://copier.readthedocs.io/) template for a simple Python projects
-
-- Python project with `src` and `test` managed by `poetry`
-- `notebooks` for Jupyter notebooks
-- `Makefile` as task runner. See `make help`
-
-## Prerequisites
-
-- `pyenv`
-- `poetry`
-- `copier`
-
-## Project Setup
-
-```sh
-$ copier copy gh:leoshimo/copier_python_template my_project
-$ cd my_project
-$ make setup
-```
-
-## Running Tasks
-
-See `make help` for list of tasks.
-
-## API Keys
-
-Add `.env` with `SOME_API_KEY=API_KEY`. Then use `python-dotenv`:
-
-```python
-import os
-from dotenv import load_dotenv
-load_dotenv()
-os.environ.get('SOME_API_KEY')
-```
-
+Hello world!

--- a/src/{{ project_name }}/app.py.jinja
+++ b/src/{{ project_name }}/app.py.jinja
@@ -1,8 +1 @@
-from fastapi import FastAPI
-from {{ project_name }}.hello import hello_world
-
-app = FastAPI()
-
-@app.get("/")
-async def index():
-    return { "message": hello_wo

# Tool Argument Parsing

In [15]:
msg = agent.messages[8]
msg

{'role': 'assistant',
 'content': 'OBSERVATION: The main.py file contains the FastAPI application and a root endpoint.\nTHOUGHT: I should append to the file using the write tool with the existing contents and the new endpoint code.\nACTION: write ["main.py", \'from fastapi import FastAPI\\napp = FastAPI()\\n\\n@app.get("/")\\nasync def root():\\n    return {"message": "Hello World"}\\n\\n@app.get("/api/diff_me")\\nasync def diff_me(githubUrl: str, prompt: str):\\n    return {"githubUrl": githubUrl, "prompt": prompt}\']'}

In [21]:
import tinygen.extract
import ast

action = tinygen.extract.extract_action(msg)
action
lst = ast.literal_eval(action.arg)

['main.py',
 'from fastapi import FastAPI\napp = FastAPI()\n\n@app.get("/")\nasync def root():\n    return {"message": "Hello World"}\n\n@app.get("/api/diff_me")\nasync def diff_me(githubUrl: str, prompt: str):\n    return {"githubUrl": githubUrl, "prompt": prompt}']

In [73]:
print(str(args[1]))

"from fastapi import FastAPI\napp = FastAPI()\n\n@app.get(\"


In [45]:
import shlex


eval(action.arg)

'main.pyfrom fastapi import FastAPI\napp = FastAPI()\n\n@app.get("/")\nasync def root():\n    return {"message": "Hello World"}\n\n@app.get("/api/diff_me")\nasync def diff_me(githubUrl: str, prompt: str):\n    return {"githubUrl": githubUrl, "prompt": prompt}'

In [44]:
shlex.get_token()

'"from fastapi import FastAPI\\napp = FastAPI()\\n\\n@app.get(\\"'

# Workspace Tools

In [16]:
import tinygen.workspace


t = tinygen.workspace.tools(wkspc)
t

NameError: name 'wkspc' is not defined